## Convolutional Neural Networks

1. Introduction to Colab.

2. Task: Implement and train a CNN on MNIST and CIFAR10 datasets using keras and GPU.

Datasets: datasets/mnist.npz and datasets/cifar10.npz

In [ ]:
!wget https://www.mimuw.edu.pl/~bilinski/VRNN2020/labs/datasets/mnist.npz

--2020-11-18 14:35:21--  https://www.mimuw.edu.pl/~bilinski/VRNN2020/labs/datasets/mnist.npz
Resolving www.mimuw.edu.pl (www.mimuw.edu.pl)... 193.0.96.14
Connecting to www.mimuw.edu.pl (www.mimuw.edu.pl)|193.0.96.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444640894 (424M)
Saving to: ‘mnist.npz’

mnist.npz           100%[===================>] 424.04M  6.85MB/s    in 55s     

2020-11-18 14:36:18 (7.66 MB/s) - ‘mnist.npz’ saved [444640894/444640894]



In [ ]:
import tensorflow as tf
!nvidia-smi
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Running on TPU  ['10.45.203.234:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.45.203.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.45.203.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
from __future__ import print_function


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

import numpy as np
data = np.load('mnist.npz')
Xtr,Ytr,Xte,Yte = data['Xtr'],data['Ytr'],data['Xte'],data['Yte']

if K.image_data_format() == 'channels_first':
    Xtr, Xte, input_shape = Xtr.reshape(Xtr.shape[0], 1, 28, 28), Xte.reshape(Xte.shape[0], 1, 28, 28), (1, 28, 28)
else:
    Xtr, Xte, input_shape = Xtr.reshape(Xtr.shape[0], 28, 28, 1), Xte.reshape(Xte.shape[0], 28, 28, 1), (28, 28, 1)
print(Xtr.shape[0], 'train samples and', Xte.shape[0], 'test samples')

print(input_shape)
with tpu_strategy.scope():
  # creating the model in the TPUStrategy scope means we will train the model on the TPU

  model = Sequential()
  model.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape=input_shape, activation='relu'))#TODO)  # conv 3x3, 32, relu
  model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))  # conv 3x3, 64, relu
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))  # max pooling 2x2
  model.add(tf.keras.layers.Dropout(0.25))  # dropout 0.25
  model.add(tf.keras.layers.Flatten())  # flatten
  model.add(tf.keras.layers.Dense(128, activation='relu'))  # dense 128, relu
  model.add(tf.keras.layers.Dropout(0.5))  # dropout 0.5
  model.add(tf.keras.layers.Dense(10, activation='softmax'))  # dense 10, softmax
  model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

  model.summary()
  model.fit(Xtr, Ytr, batch_size=1024, epochs=20, validation_data=(Xte, Yte))
  score = model.evaluate(Xte, Yte, batch_size=128)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])

60000 train samples and 10000 test samples
(28, 28, 1)
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               1179776   
_________________________________________________________________

59/59 [==============================] - ETA: 0s - loss: 0.9528 - accuracy: 0.7264WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0015s vs `on_test_batch_end` time: 0.0097s). Check your callbacks.


59/59 [==============================] - 6s 96ms/step - loss: 0.9528 - accuracy: 0.7264 - val_loss: 0.2755 - val_accuracy: 0.9180
Epoch 2/20
59/59 [==============================] - 2s 31ms/step - loss: 0.3178 - accuracy: 0.9047 - val_loss: 0.1808 - val_accuracy: 0.9445
Epoch 3/20
59/59 [==============================] - 2s 27ms/step - loss: 0.2326 - accuracy: 0.9319 - val_loss: 0.1236 - val_accuracy: 0.9627
Epoch 4/20
59/59 [==============================] - 2s 27ms/step - loss: 0.1761 - accuracy: 0.9489 - val_loss: 0.0951 - val_accuracy: 0.9711
Epoch 5/20
59/59 [==============================] - 2s 27ms/step - loss: 0.1401 - accuracy: 0.9585 - val_loss: 0.0717 - val_accuracy: 0.9772
Epoch 6/20
59/59 [==============================] - 2s 27ms/step - loss: 0.1110 - accuracy: 0.9676 - val_loss: 0.0643 - val_accuracy: 0.9802
Epoch 7/20
59/59 [==============================] - 2s 27ms/step - loss: 0.0972 - accuracy: 0.9714 - val_loss: 0.0543 - val_accuracy: 0.9836
Epoch 8/20
59/59 [======

79/79 [==============================] - 2s 30ms/step - loss: 0.0304 - accuracy: 0.9903
Test loss: 0.0303773432970047
Test accuracy: 0.9902999997138977


In [ ]:
tf.test.gpu_device_name()
print(data['Xtr'].shape)

(60000, 784)


In [ ]:
!wget https://www.mimuw.edu.pl/~bilinski/VRNN2020/labs/datasets/cifar10.npz

--2020-11-18 14:17:50--  https://www.mimuw.edu.pl/~bilinski/VRNN2020/labs/datasets/cifar10.npz
Resolving www.mimuw.edu.pl (www.mimuw.edu.pl)... 193.0.96.14
Connecting to www.mimuw.edu.pl (www.mimuw.edu.pl)|193.0.96.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 742080894 (708M)
Saving to: ‘cifar10.npz’

cifar10.npz         100%[===================>] 707.70M  4.15MB/s    in 2m 21s  

2020-11-18 14:20:12 (5.00 MB/s) - ‘cifar10.npz’ saved [742080894/742080894]



In [ ]:
data = np.load('cifar10.npz')
Xtr,Ytr,Xte,Yte = data['Xtr'],data['Ytr'],data['Xte'],data['Yte']

print(Xtr.shape)
if K.image_data_format() == 'channels_first':
    Xtr, Xte, input_shape = Xtr.reshape(Xtr.shape[0], 3, 32, 32), Xte.reshape(Xte.shape[0], 3, 32, 32), (3, 32, 32)
else:
    Xtr, Xte, input_shape = Xtr.reshape(Xtr.shape[0], 32, 32, 3), Xte.reshape(Xte.shape[0], 32, 32, 3), (32, 32, 3)
print(Xtr.shape[0], 'train samples and', Xte.shape[0], 'test samples')

model = Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape=input_shape, activation='relu'))#TODO)  # conv 3x3, 32, relu
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))  # conv 3x3, 64, relu
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))  # max pooling 2x2
model.add(tf.keras.layers.Dropout(0.25))  # dropout 0.25
model.add(tf.keras.layers.Flatten())  # flatten
model.add(tf.keras.layers.Dense(128, activation='relu'))  # dense 128, relu
model.add(tf.keras.layers.Dropout(0.5))  # dropout 0.5
model.add(tf.keras.layers.Dense(10, activation='softmax'))  # dense 10, softmax
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=tensorflow.keras.optimizers.Adam(), metrics=['accuracy'])
model.summary()
model.fit(Xtr, Ytr, batch_size=128, epochs=20, validation_data=(Xte, Yte))

model.fit(Xtr, Ytr, batch_size=128, epochs=20, validation_data=(Xte, Yte))
score = model.evaluate(Xte, Yte, batch_size=128)

(50000, 3072)
50000 train samples and 10000 test samples
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               1605760   
______________________________________________________________